In [1]:
from pandas import read_csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as mpl
import math 
import seaborn as sea
import statistics
from scipy import stats
from numpy import percentile
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report 
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_boston
import matplotlib
import statsmodels.api as sm
%matplotlib inline
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
import warnings

In [2]:
def accuracyscore(X,Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)  
    warnings.simplefilter('ignore')
    LR = LogisticRegression()
    KNN = KNeighborsClassifier()
    NB = GaussianNB()
    LSVM = LinearSVC()
    NLSVM = SVC(kernel='rbf')
    DT = DecisionTreeClassifier()
    RF = RandomForestClassifier()
    #
    LR_fit = LR.fit(X_train, Y_train)
    KNN_fit = KNN.fit(X_train, Y_train)
    NB_fit = NB.fit(X_train, Y_train)
    LSVM_fit = LSVM.fit(X_train, Y_train)
    NLSVM_fit = NLSVM.fit(X_train, Y_train)
    DT_fit = DT.fit(X_train, Y_train)
    RF_fit = RF.fit(X_train, Y_train)
    #
    LR_pred = LR_fit.predict(X_test)
    KNN_pred = KNN_fit.predict(X_test)
    NB_pred = NB_fit.predict(X_test)
    LSVM_pred = LSVM_fit.predict(X_test)
    NLSVM_pred = NLSVM_fit.predict(X_test)
    DT_pred = DT_fit.predict(X_test)
    RF_pred = RF_fit.predict(X_test)
    #
    print("Logistic Regression is %f percent accurate" % (accuracy_score(LR_pred, Y_test)*100))
    #
    print("KNN is %f percent accurate" % (accuracy_score(KNN_pred, Y_test)*100))
    #
    print("Naive Bayes is %f percent accurate" % (accuracy_score(NB_pred, Y_test)*100))
    #
    print("Linear SVMs is %f percent accurate" % (accuracy_score(LSVM_pred, Y_test)*100))
    #
    print("Non Linear SVMs is %f percent accurate" % (accuracy_score(NLSVM_pred, Y_test)*100))
    #
    print("Decision Trees is %f percent accurate" % (accuracy_score(DT_pred, Y_test)*100))
    #
    print("Random Forests is %f percent accurate" % (accuracy_score(RF_pred, Y_test)*100))

In [3]:
bank = read_csv('BankChurnML.csv')
del bank["Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"]
del bank["Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1"]
X = bank.drop("Attrition_Flag",1)    
Y = bank["Attrition_Flag"] 

# Backward Elimination

In [4]:
#Adding constant column of ones, mandatory for sm.OLS model
X = bank.drop("Attrition_Flag",1)    #Feature Matrix
Y = bank["Attrition_Flag"]           #Target Variable

X_1 = sm.add_constant(X)
#Fitting sm.OLS model
model = sm.OLS(Y,X_1).fit()

In [5]:
cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(Y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
X2 = bank[selected_features_BE]
accuracyscore(X2,Y)

Logistic Regression is 89.826041 percent accurate
KNN is 86.188719 percent accurate
Naive Bayes is 87.295730 percent accurate
Linear SVMs is 89.562467 percent accurate
Non Linear SVMs is 89.246178 percent accurate
Decision Trees is 92.409067 percent accurate
Random Forests is 95.044808 percent accurate


# Recursive Feature Elimination

In [10]:
X = bank.drop("Attrition_Flag",1)    #Feature Matrix
y = bank["Attrition_Flag"]           #Target Variable

model = LinearRegression()
#Initializing RFE model
rfe = RFE(model, 7)
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)

#no of features
nof_list=np.arange(1,20)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

cols = list(X.columns)
model = LinearRegression()
#Initializing RFE model
rfe = RFE(model, 10)             
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)              
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

X3 = bank[selected_features_rfe]
print(X3.shape)
accuracyscore(X3,Y)

Optimum number of features: 16
Score with 16 features: 0.374021
Index(['Gender', 'Months_on_book', 'Total_Relationship_Count',
       'Months_Inactive_12_mon', 'Contacts_Count_12_mon',
       'Total_Revolving_Bal', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1'],
      dtype='object')
(9485, 10)
Logistic Regression is 91.407486 percent accurate
KNN is 91.249341 percent accurate
Naive Bayes is 88.877174 percent accurate
Linear SVMs is 91.618345 percent accurate
Non Linear SVMs is 90.774908 percent accurate
Decision Trees is 94.359515 percent accurate
Random Forests is 96.731682 percent accurate
